In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import sys

from torch.utils.data import DataLoader
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
import json
import glob
import os
from PIL import Image
import random

import bottleneck
from resnet_pytorch import ResNet

sys.path.insert(0, '../')
from datasets import load_dataset, load_metric, Dataset, DatasetDict

## Pre-processing data

### Example of processing labels

In [2]:
# load .env file
from dotenv import load_dotenv
load_dotenv()

#loading labels
DATA_PATH = os.getenv('DATA_PATH')
files = glob.glob(DATA_PATH + "*.json")
number_of_files = 20

labels = []  # Initialize a list to store data from all files

if not files:
    print("No files found. Check the directory path.")
else:
    for file_path in files[:number_of_files]:
        try:
            with open(file_path, 'r') as file:
                # Load the content of the current file
                file_data = json.load(file)
                if "coordinates" in file_data:
                    labels.append(file_data)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON from {file_path}: {e}")
        except Exception as e:
            print(f"Error opening or reading {file_path}: {e}")

# Output how many files were successfully loaded
print(f"Loaded data from {len(labels)} files.")


Loaded data from 19 files.


In [3]:
labels

[{'coordinates': [51.08673095703125, 3.8212528228759766]},
 {'coordinates': [38.69101333618164, 22.3980770111084]},
 {'coordinates': [48.94839096069336, -55.669677734375]},
 {'coordinates': [45.62511444091797, -79.88330841064453]},
 {'coordinates': [55.164520263671875, 61.39071273803711]},
 {'coordinates': [48.70686340332031, 2.33329176902771]},
 {'coordinates': [57.00322341918945, 8.783960342407227]},
 {'coordinates': [56.090667724609375, 37.55679702758789]},
 {'coordinates': [17.896394729614258, 102.7203598022461]},
 {'coordinates': [56.845924377441406, 60.62533187866211]},
 {'coordinates': [64.14692687988281, -21.88874053955078]},
 {'coordinates': [8.691835403442383, 80.51228332519531]},
 {'coordinates': [-30.146350860595703, -51.12898635864258]},
 {'coordinates': [50.983604431152344, -114.00588989257812]},
 {'coordinates': [40.71820831298828, -73.95831298828125]},
 {'coordinates': [56.130924224853516, 47.227935791015625]},
 {'coordinates': [-37.798309326171875, 145.2571258544922]},

In [4]:
# Just load the first image, avoid Multiplayer
labels[0]["coordinates"]

[51.08673095703125, 3.8212528228759766]

### Example of processing input images

In [5]:
# loading images
files = glob.glob(DATA_PATH + "*.png")

if not files:
    print("No files found. Check the directory path.")
else:
    images = [] 
    for file_path in files[:number_of_files]:
        try:
            # channels, height, width is the pytorch convention
            with Image.open(file_path) as img:
                img_array = np.array(img)
                img_array = img_array.transpose((2, 0, 1))
                #images.append(img_array)
                images.append(file_path)
        except IOError as e:
            print(f"Error opening or reading {file_path}: {e}")

# Output how many files were successfully processed
print(f"Processed {len(images)} image files.")
#display(Image.open(images[0]))

Processed 20 image files.


In [6]:
# Inside the loop, convert the image to a numpy array and store it
with Image.open(file_path) as img:
    width, height = img.size
    img_array = np.array(img)
    print(f"Image size (height, width): {height}x{width}")
    print(f"Image array shape: {img_array.shape}")
    print(f"Image mode: {img.mode}")
    print(f"Image format: {img.format}")

Image size (height, width): 180x320
Image array shape: (180, 320, 4)
Image mode: RGBA
Image format: PNG


In [7]:
#images[0].shape
images[0]

'../.data/geoguessr_location_singleplayer_rm8goISVT6SLjGR9_4_resized.png'

### Class of creating dataset

In [8]:
class CustomImageDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)

        return image, label

# Define transformations
transform = transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [9]:
# split before to train, validation, and test
dataset = CustomImageDataset(images, labels, transform=transform)
loader = DataLoader(dataset, batch_size=2, shuffle=True)

In [10]:
loader

In [11]:
loader.dataset[0][0].shape

torch.Size([3, 100, 100])

## new code

In [12]:
import json
import sys
import os
sys.path.insert(0, '../')
from data_loader import get_data_to_load

# loading local data
DATA_PATH = os.getenv('DATA_PATH')
if not DATA_PATH:
    raise ValueError("DATA_PATH environment variable is not set")

files = glob.glob(DATA_PATH + "*.json")

# get list with local data and file paths
list = get_data_to_load(loading_file='./data_list', file_location=DATA_PATH, image_file_location=DATA_PATH, allow_new_file_creation=True, from_remote_only=True, download_link='env', limit=100, shuffle_seed=43)

# List of data to load
print(len(list))

Getting files list from remote
Got files list from remote
Parsed files list from remote
All remote files: 274796
Filtering out unpaired files
Filtered out 17666 unpaired files
Relevant files: 257130
Limited files: 200
200


In [36]:
input_images = [item for item in list if item.endswith(".png")]
labels = [item for item in list if item.endswith(".json")]

In [39]:
input_images[0]

'../.data/geoguessr_location_singleplayer_3jd33hbXzAHBrYDj_4.png'

In [49]:

# Adding your custom data loading script's path to the system path
sys.path.insert(0, '../')
from data_loader import get_data_to_load

# Assume DATA_PATH is set properly
DATA_PATH = os.getenv('DATA_PATH')
if not DATA_PATH:
    raise ValueError("DATA_PATH environment variable is not set")

# Get all JSON files in the directory
json_files = glob.glob(os.path.join(DATA_PATH, "*.json"))
# Assuming corresponding PNG files should exist
png_files = [f.replace('.json', '.png') for f in json_files]

# Get list with local data and file paths from your custom loader
loaded_files = get_data_to_load(loading_file='./data_list', file_location=DATA_PATH, image_file_location=DATA_PATH, allow_new_file_creation=True, from_remote_only=True, download_link='env', limit=10, shuffle_seed=43)

# Checking the existence of each file in the loaded list
missing_files = [f for f in loaded_files if not os.path.exists(f)]
if missing_files:
    print(f"Some files are missing: {missing_files}")
else:
    print("All files from the loaded list exist in the directory.")

Getting files list from remote
Got files list from remote
Parsed files list from remote
All remote files: 274796
Filtering out unpaired files
Filtered out 17666 unpaired files
Relevant files: 257130
Limited files: 20
Some files are missing: ['../.data/geoguessr_location_singleplayer_2V749xmsAtz2islo_2.png', '../.data/geoguessr_location_singleplayer_TH7jVALVRMmGsPdr_1.png', '../.data/geoguessr_location_singleplayer_I5yMnDSVlaGLEa12_1.png', '../.data/geoguessr_location_singleplayer_NLPqsR0Hd0a0qQp8_0.png', '../.data/geoguessr_location_singleplayer_hszkmUG9P9AhshF0_3.png', '../.data/geoguessr_location_singleplayer_yfDqCnA3F4irzZ99_0.png', '../.data/geoguessr_location_singleplayer_hk06fSN25XgNQ58S_3.png', '../.data/geoguessr_location_singleplayer_gDZBXUA5tdrGkN8q_0.png', '../.data/geoguessr_location_singleplayer_91rU5yOIkTwHiYtQ_0.png', '../.data/geoguessr_location_singleplayer_lys6kNZhPdxRYUXU_4.png']


In [50]:
len(missing_files)

10

In [51]:
# path from the "combined" data set from yutaro
png_files[0]

'../.data/geoguessr_result_singleplayer_p2oxCfgo7krY3o3k_3.png'

In [52]:
# path from the data_loader with the cleaned images
input_images[0]

'../.data/geoguessr_location_singleplayer_3jd33hbXzAHBrYDj_4.png'

In [44]:
# example of loading data for input images
with Image.open(input_images[40]) as img:
  img_array = np.array(img)
  img_array = img_array.transpose((2, 0, 1))
  #images.append(img_array)
  images.append(file_path)

FileNotFoundError: [Errno 2] No such file or directory: '../.data/geoguessr_location_singleplayer_Nj7x19xz68DjgyJK_4.png'

In [38]:
# example of loading data for labels
with open(labels[0], 'r') as file:
      file_data = json.load(file)
      label = file_data['coordinates'] if 'coordinates' in file_data else None
      print(label)

[15.19734001159668, 120.5780029296875]


In [13]:
list[0]

'../.data/geoguessr_result_singleplayer_3jd33hbXzAHBrYDj_4.json'

In [16]:
import torch
import json
from torchvision import transforms
from torch.utils.data import Dataset
from PIL import Image
import glob
import numpy as np

class CustomImageDataset(Dataset):
    def __init__(self, image_paths, json_paths, transform=None):
        self.image_paths = image_paths
        self.json_paths = json_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # Load image
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        # Load labels from JSON file
        json_path = self.json_paths[idx]
        with open(json_path, 'r') as file:
            file_data = json.load(file)
            label = file_data['coordinates'] if 'coordinates' in file_data else None

        return image, label

# Define transformations
transform = transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [17]:
input_images = [item for item in list if item.endswith('.png')]
labels = [item for item in list if item.endswith('.json')]

assert len(input_images) == len(labels), "Mismatch in number of images and labels"

In [27]:
labels.__getitem__(5)

'../.data/geoguessr_result_singleplayer_VROl268f9c1R4jWy_0.json'

In [19]:
dataset = CustomImageDataset(input_images, labels, transform=transform)
loader = DataLoader(dataset, batch_size=2, shuffle=True)

In [23]:
dataset.image_paths[0]

'../.data/geoguessr_location_singleplayer_3jd33hbXzAHBrYDj_4.png'

In [ ]:
# Creating the dataset and dataloader
dataset = CustomImageDataset(input_images, labels, transform=transform)
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

In [20]:
for images, labels in loader:
    print("Images batch shape:", images.shape)
    print("Labels batch shape:", labels.shape)
    break  # After printing the first batch, exit the loop

FileNotFoundError: [Errno 2] No such file or directory: '../.data/geoguessr_location_singleplayer_NVY1TBOqwZCElNOL_2.png'

In [ ]:
# Preload images and labels into tensors
images = []
labels_data = []

for img_path, json_path in zip(input_images, labels):
    # Load and transform the image
    image = Image.open(img_path).convert('RGB')
    image = transform(image)
    images.append(image)

    # Load and process the JSON file for labels
    with open(json_path, 'r') as file:
        file_data = json.load(file)
        label = file_data['coordinates'] if 'coordinates' in file_data else [0, 0]  # Default to [0,0] if no coordinates
        labels_data.append(torch.tensor(label, dtype=torch.float32))

# Create dataset from preloaded data
class CustomImageDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

# Initialize dataset and DataLoader
dataset = CustomImageDataset(images, labels_data)
loader = DataLoader(dataset, batch_size=2, shuffle=True)

In [ ]:
# Example usage of the DataLoader
for images, labels in loader:
    print(images.shape, labels)
    break  # Break after printing first batch for demonstration

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, images, labels):
        """
        Args:
            images (list of torch.Tensor): List of images as tensors.
            labels (list): List of labels.
        """
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        # Retrieve image and label already in the correct format
        image = self.images[idx]
        label = self.labels[idx]

        return image, label

In [ ]:
from torchvision import transforms
from PIL import Image
import glob
import numpy as np

# Define transformations
transform = transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load images
image_paths = glob.glob('../../1_data_collection/.data/*.png')
images = []
for img_path in image_paths:
    image = Image.open(img_path).convert('RGB')
    image = transform(image)
    images.append(image)

# Assume labels are loaded and processed into a corresponding list `labels`
labels = [...]  # This should match the length and order of `images`

# Create dataset
dataset = CustomImageDataset(images, labels)

In [ ]:
import glob
import json

# Path where the JSON files are stored
json_path = '../../1_data_collection/.data/*.json'
files = glob.glob(json_path)
number_of_files = 20  # Limit the number of files to process

labels = []  # Initialize a list to store labels from the files

if not files:
    print("No files found. Check the directory path.")
else:
    for file_path in files[:number_of_files]:
        try:
            with open(file_path, 'r') as file:
                # Load the content of the current file
                file_data = json.load(file)
                # Check if 'coordinates' key exists
                if "coordinates" in file_data:
                    # Append the coordinates or the whole data, depending on need
                    labels.append(file_data['coordinates'])
                else:
                    # Handle cases where 'coordinates' might not exist
                    labels.append(None)  # Or some default value, e.g., {}
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON from {file_path}: {e}")
        except Exception as e:
            print(f"Error opening or reading {file_path}: {e}")

# Output how many files were successfully loaded with labels
print(f"Loaded labels from {len(labels)} files.")


### Splitting dataset into training, validation and testing sets

In [ ]:
# create dataset
data = {}
data["input"] = labels

# Define split ratios
train_ratio = 0.7
val_ratio = 0.20
test_ratio = 0.10

# Shuffle data randomly
random.shuffle(data["input"])

# Calculate split indices
total_images = len(data["input"])
train_end = int(train_ratio * total_images)
val_end = train_end + int(val_ratio * total_images)

# Split the data
train_data = data["input"][:train_end]
val_data = data["input"][train_end:val_end]
test_data = data["input"][val_end:]

print(f"Train set size: {len(train_data)}")
print(f"Validation set size: {len(val_data)}")
print(f"Test set size: {len(test_data)}")


In [ ]:
#Loading dataet and crearting, split before!!!
dataset = DatasetDict({
    "train": train_data,
    "validation": val_data,
    "test": test_data
})

In [ ]:
dataset

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    #transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

### Creating Dataloaders

In [ ]:
train = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)

trainloader = torch.utils.data.DataLoader(train, batch_size=128, shuffle=True, num_workers=2)

test = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

testloader = torch.utils.data.DataLoader(test, batch_size=128,shuffle=False, num_workers=2)

## Model

In [ ]:
model = ResNet.from_pretrained('resnet18', num_classes=2)

In [ ]:
#print(model)

## Training

In [ ]:
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
net = ResNet50(10).to('cuda')

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.1, patience=5)

In [ ]:
EPOCHS = 200
for epoch in range(EPOCHS):
    losses = []
    running_loss = 0
    for i, inp in enumerate(trainloader):
        inputs, labels = inp
        inputs, labels = inputs.to('cuda'), labels.to('cuda')
        optimizer.zero_grad()
    
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if i%100 == 0 and i > 0:
            print(f'Loss [{epoch+1}, {i}](epoch, minibatch): ', running_loss / 100)
            running_loss = 0.0

    avg_loss = sum(losses)/len(losses)
    scheduler.step(avg_loss)
            
print('Training Done')